In [68]:
from lattedb.quarksmear.models import GaugeCovariantGaussian as quarksmear_GaugeCovariantGaussian
from lattedb.fermionaction.models import Hisq as fermionaction_Hisq
from lattedb.gaugeaction.models import LuescherWeisz as gaugeaction_LuescherWeisz
from lattedb.fermionaction.models import MobiusDW as fermionaction_MobiusDW
from lattedb.gaugeconfig.models import Nf211 as gaugeconfig_Nf211
from lattedb.quarksmear.models import Point as quarksmear_Point
from lattedb.linksmear.models import Unsmeared as linksmear_Unsmeared
from lattedb.linksmear.models import WilsonFlow as linksmear_WilsonFlow
from lattedb.propagator.models import OneToAll as propagator_OneToAll

In [69]:
import sys
sys.path.append('../../nucleon_elastic_FF/scripts/area51_files')
sys.path.append('../../nucleon_elastic_FF/scripts')
import sources
import utils

In [70]:
import a09m310 as a51
p = a51.mpirun_params("summit")
p["STREAM"] = "e"

In [71]:
cfgs_run, p['srcs'] = utils.parse_cfg_src_argument([p["cfg_i"], p["cfg_f"], p["cfg_d"]],'',p)

In [72]:
gaugeconfigs = gaugeconfig_Nf211.objects.filter(short_tag=p["ENS_ABBR"])

In [73]:
linksmear_tag = f"wflow{p['FLOW_TIME']}_n{p['FLOW_STEP']}"
fermionaction_linksmear, created = linksmear_WilsonFlow.objects.get_or_create(
	flowtime=p["FLOW_TIME"], # Flow time in lattice units
	flowstep=p["FLOW_STEP"], # Number of diffusion steps
	tag=linksmear_tag, # (Optional) User defined tag for easy searches
)
print(created)

smear_tag = f"wv_w{p['WF_S']}_n{p['WF_N']}"
gaussian, created = quarksmear_GaugeCovariantGaussian.objects.get_or_create(
	radius=p["WF_S"], # Smearing radius in lattice units
	step=p["WF_N"], # Number of smearing steps
	tag= smear_tag, # (Optional) User defined tag for easy searches
	description="Gaussian smearing.", # (Optional) Description of the quark smearing operator
)
print(created)

point, created = quarksmear_Point.objects.get_or_create(
	tag="", # (Optional) User defined tag for easy searches
	description="Dirac delta function.", # (Optional) Description of the quark smearing operator
)
print(created)
smearings = [('SS', gaussian), ('PS', point)]

False
False
False


In [77]:
quark_list = [("light", p["MV_L"]), ("strange", p["MV_S"])]
for quark in quark_list:
    fermion_tag = f"M5{p['M5']}_L5{p['L5']}_a{p['alpha5']}"
    fermionaction, created = fermionaction_MobiusDW.objects.get_or_create(
        quark_mass=quark[1], # Input quark mass
        quark_tag=quark[0], # Type of quark
        l5=p["L5"], # Length of 5th dimension
        m5=p["M5"], # 5th dimensional mass
        b5=p["B5"], # Mobius kernel parameter [a5 = b5 - c5, alpha5 * a5 =…
        c5=p["C5"], # Mobius kernal perameter
        linksmear=fermionaction_linksmear, # Foreign Key pointing to additional gauge `linksmear` outside of Monte Carlo.
        tag=fermion_tag, # (Optional) User defined tag for easy searches
    )
    print("fermionaction:", created)
    for smearing in smearings:
        for gaugeconfig in gaugeconfigs:
            for source in p['srcs'][gaugeconfig.config]:
                xyzt = sources.xyzt(source)
                prop_tag = f"prop_{smearing[0]}_mq{quark[1]}_{linksmear_tag}_{fermion_tag}_cfg_{gaugeconfig.config}_src{source}_{smear_tag}"
                propagator_onetoall, created = propagator_OneToAll.objects.update_or_create(
                    gaugeconfig=gaugeconfig, # Foreign Key pointing to specific \texttt{gaugeconfig} inverted on
                    fermionaction=fermionaction, # Foreign Key pointing to valence lattice `fermionaction`. This is the valence action.
                    origin_x=int(xyzt[0]), # x-coordinate origin location of the propagator
                    origin_y=int(xyzt[1]), # y-coordinate origin location of the propagator
                    origin_z=int(xyzt[2]), # z-coordinate origin location of the propagator
                    origin_t=int(xyzt[3]), # t-coordinate origin location of the propagator
                    sourcesmear=gaussian, # Foreign Key pointing to source `quarksmear`
                    sinksmear=smearing[1], # Foreign Key pointing to sink `quarksmear`
                    tag=prop_tag, # (Optional) User defined tag for easy searches
                )
                print(created, prop_tag)

fermionaction: False
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_306_srcx28y2z23t62_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_306_srcx12y18z7t62_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_306_srcx21y30z16t14_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_306_srcx5y14z0t14_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_306_srcx12y6z29t86_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_306_srcx28y22z13t86_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_306_srcx4y24z25t38_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_306_srcx20y8z9t38_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_312_srcx17y18z3t40_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_312_srcx1y2z19t40_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_312_srcx11y3z11t88_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n4

True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_384_srcx15y26z21t82_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_384_srcx24y10z4t34_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_384_srcx8y26z20t34_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_384_srcx12y20z12t10_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_384_srcx28y4z28t10_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_384_srcx21y8z18t58_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_384_srcx5y24z2t58_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_390_srcx11y19z15t44_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_390_srcx27y3z31t44_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_390_srcx27y15z29t92_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_390_srcx11y31z13t92_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5

True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_456_srcx1y5z21t47_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_456_srcx26y7z6t95_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_456_srcx10y23z22t95_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_462_srcx12y23z9t41_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_462_srcx28y7z25t41_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_462_srcx30y5z13t89_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_462_srcx14y21z29t89_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_462_srcx1y5z2t65_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_462_srcx17y21z18t65_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_462_srcx0y16z30t17_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_462_srcx16y0z14t17_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_

True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_534_srcx0y18z16t48_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_534_srcx1y18z28t0_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_534_srcx17y2z12t0_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_540_srcx3y16z5t43_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_540_srcx19y0z21t43_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_540_srcx29y3z6t91_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_540_srcx13y19z22t91_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_540_srcx0y7z30t67_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_540_srcx16y23z14t67_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_540_srcx26y3z28t19_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_540_srcx10y19z12t19_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_5

True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_618_srcx21y27z22t15_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_618_srcx3y24z27t63_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_618_srcx19y8z11t63_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_618_srcx5y3z16t39_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_618_srcx21y19z0t39_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_618_srcx7y14z6t87_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_618_srcx23y30z22t87_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_624_srcx30y9z13t50_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_624_srcx14y25z29t50_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_624_srcx13y0z17t2_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_624_srcx29y16z1t2_wv_w3.5_n45
True prop_SS_mq0.00951_wflow1.0_n40_M51.1_L56_a1.5_cfg_

...

True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4662_srcx4y11z22t91_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4662_srcx20y27z6t91_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4662_srcx16y15z21t43_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4662_srcx0y31z5t43_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4662_srcx9y3z17t19_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4662_srcx25y19z1t19_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4662_srcx10y15z0t67_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4662_srcx26y31z16t67_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4674_srcx25y29z7t50_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4674_srcx9y13z23t50_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4674_srcx28y0z19t2_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4

True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4740_srcx15y12z28t30_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4740_srcx31y28z12t30_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4740_srcx13y30z23t6_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4740_srcx29y14z7t6_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4740_srcx19y24z7t54_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4740_srcx3y8z23t54_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4746_srcx4y22z13t54_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4746_srcx20y6z29t54_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4746_srcx21y21z31t6_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4746_srcx5y5z15t6_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4746_srcx30y15z26t78_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4

True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4812_srcx20y2z24t60_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4812_srcx4y18z8t60_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4818_srcx22y22z7t6_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4818_srcx6y6z23t6_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4818_srcx2y16z5t54_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4818_srcx18y0z21t54_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4818_srcx0y17z5t30_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4818_srcx16y1z21t30_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4818_srcx28y2z27t78_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4818_srcx12y18z11t78_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4824_srcx6y6z15t58_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4824_s

True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4890_srcx22y26z21t14_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4890_srcx6y10z5t14_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4890_srcx24y16z1t62_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4890_srcx8y0z17t62_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4896_srcx26y22z22t61_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4896_srcx10y6z6t61_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4896_srcx13y31z29t13_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4896_srcx29y15z13t13_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4896_srcx3y26z27t85_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4896_srcx19y10z11t85_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4896_srcx11y0z7t37_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg

True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4968_srcx23y3z20t7_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4968_srcx7y19z4t7_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4968_srcx23y12z5t55_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4968_srcx7y28z21t55_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4968_srcx28y29z3t31_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4968_srcx12y13z19t31_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4968_srcx7y16z21t79_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4968_srcx23y0z5t79_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4974_srcx11y3z25t46_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4974_srcx27y19z9t46_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_4974_srcx28y7z21t94_wv_w3.5_n45
True prop_PS_mq0.0491_wflow1.0_n40_M51.1_L56_a1.5_cfg_497